### Predicting Mask on Augmented Dataset

1. Load Model
2. Load Test Dataset - Images
3. Predict Mask
4. Calculate FPS

In [2]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

#### Seeding

In [5]:
# Seeding
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

#### Hyperparameters

In [4]:
# Hyperparameters
height = 768
width = 512
num_classes = 3

#### Path

In [ ]:
# Paths
dataset_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/processed_dataset/aug/test"
save_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/predicted_masks/aug"
model_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/files/aug/unet-multiclass.h5"

#### Create Folder

In [ ]:
os.makedirs(save_path, exist_ok=True)

#### Load Model

In [ ]:
# Load Model
model = tf.keras.models.load_model(model_path)

In [ ]:
# model.summary()

#### Load Test Dataset: Images

In [ ]:
# Define color map for visualization (BGR format)
colors = {
    0: (0, 0, 0),       # Non-Drivable Area (black)
    1: (79, 247, 211),  # My Way (#d3f74f) in BGR
    2: (247, 93, 79)    # Other Way (#4f5df7) in BGR
}

In [ ]:
# Load test images
test_x = sorted(glob(os.path.join(dataset_path, "images", "*")))

#### Predict the mask and calculate FPS

In [ ]:
time_taken = []

for x_path in tqdm(test_x, desc="Predicting Masks"):
    name = os.path.basename(x_path)
    x = cv2.imread(x_path, cv2.IMREAD_COLOR)

    if x is None:
        print(f"Error loading image {x_path}, skipping.")
        continue

    x = cv2.resize(x, (width, height))
    x = x / 255.0
    x = np.expand_dims(x, axis=0)

    start_time = time.time()
    p = model.predict(x)[0]
    time_taken.append(time.time() - start_time)

    p = np.argmax(p, axis=-1)  # Convert probabilities to class labels

    # Create a blank color mask
    p_colored = np.zeros((height, width, 3), dtype=np.uint8)

    # Assign colors based on class labels
    for label, color in colors.items():
        p_colored[p == label] = color

    # Save predicted mask
    cv2.imwrite(os.path.join(save_path, f"{name.split('.')[0]}.png"), p_colored)

In [ ]:
# FPS Calculation
mean_time = np.mean(time_taken)
mean_fps = 1 / mean_time

In [ ]:
print(f"Mean time taken: {mean_time:.4f} seconds")
print(f"Mean FPS: {mean_fps:.2f}")